In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
#import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
from numpy import array
import datetime
import io

C:\Users\Flo\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#expected soemthing like this (e.g.): load_plot_to_tensorboard([range(len(train_data)),train_data,'k',"simple presentation of orginal data"], [same structue], ..., z)
#expected something like this (e.g.): load_plot_to_tensorboard_x_plots_y_scatter()
def load_plot_to_tensorboard_x_plots_y_scatter(plots=[],scatter=[],name="Not Specified"):
    plt.ioff()
    plt.figure(figsize=(40, 15))
    plt.title('stock market prediction - Prediction on Prediction')
    for i in plots:
        plt.plot(i[0],i[1],i[2])
    for j in scatter:
        plt.scatter(x=j[0],y=j[1],c=j[2]) 
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image=tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return tf.summary.image(name,image,max_outputs=1)

In [3]:
# data_list expected: [data_set,scaled_dataset,training_data,test_data]
def create_and_add_summaries(data_list,writer=None,session=None):
    color_original_data_plot='b'
    color_train_data_plot='g'
    color_original_data_scatter='blue'
    color_train_data_scatter='green'
    color_test_data_plot='m'
    color_test_data_scatter="magenta"
    
    summary_original_unscaled = session.run(load_plot_to_tensorboard_x_plots_y_scatter([[range(len(data_list[0])),data_list[0],color_original_data_plot]],name="UNSCALED_ORIGINAL_DATA_PLOT"))
    summary_original = session.run(load_plot_to_tensorboard_x_plots_y_scatter([[range(len(data_list[1])),data_list[1],color_original_data_plot]],name="ORIGINAL_DATA_PLOT"))
    summary_train = session.run(load_plot_to_tensorboard_x_plots_y_scatter([[range(window_size,window_size+len(data_list[2])),data_list[2],color_train_data_plot]],name="TRAIN_DATA_PLOT"))
    summary_original_with_train = session.run(load_plot_to_tensorboard_x_plots_y_scatter([[range(len(data_list[1])),data_list[1],color_original_data_plot],[range(window_size,window_size+len(data_list[2])),data_list[2],color_train_data_plot]],name="ORIGINAL_DATA_AND_TRAIN_DATA_PLOT"))
    summary_test = session.run(load_plot_to_tensorboard_x_plots_y_scatter([[range(window_size+train_range ,window_size+ train_range+ len(data_list[3])),data_list[3],color_test_data_plot]],name="TEST_DATA_PLOT"))
    summary_test_with_train = session.run(load_plot_to_tensorboard_x_plots_y_scatter([[range(window_size+train_range ,window_size+ train_range+ len(data_list[3])),data_list[3],color_test_data_plot],[range(window_size,window_size+len(data_list[2])),data_list[2],color_train_data_plot]],name="TEST_AND_TRAIN_DATA_PLOT"))
    summary_all = session.run(load_plot_to_tensorboard_x_plots_y_scatter([[range(len(data_list[1])),data_list[1],color_original_data_plot],[range(window_size,window_size+len(data_list[2])),data_list[2],color_train_data_plot],[range(window_size+train_range ,window_size+ train_range+ len(data_list[3])),data_list[3],color_test_data_plot]],name="SUMMARIZE_ALL_PLOT"))
    
    summary_original_scatter = session.run(load_plot_to_tensorboard_x_plots_y_scatter([],[[range(len(data_list[1])),data_list[1],color_original_data_scatter]],name="ORIGINAL_DATA_SCATTER"))
    summary_train_scatter = session.run(load_plot_to_tensorboard_x_plots_y_scatter([],[[range(window_size,window_size+len(data_list[2])),data_list[2],color_train_data_scatter]],name="TRAIN_DATA_SCATTER"))
    summary_original_with_train_scatter = session.run(load_plot_to_tensorboard_x_plots_y_scatter([],[[range(len(data_list[1])),data_list[1],color_original_data_scatter],[range(window_size,window_size+len(data_list[2])),data_list[2],color_train_data_scatter]],name="ORIGINAL_DATA_AND_TRAIN_DATA_SCATTER"))
    summary_test_scatter = session.run(load_plot_to_tensorboard_x_plots_y_scatter([],[[range(window_size+train_range ,window_size+ train_range+ len(data_list[3])),data_list[3],color_test_data_scatter]],name="TEST_DATA_SCATTER"))
    summary_test_with_train_scatter = session.run(load_plot_to_tensorboard_x_plots_y_scatter([],[[range(window_size+train_range ,window_size+ train_range+ len(data_list[3])),data_list[3],color_test_data_scatter],[range(window_size,window_size+len(data_list[2])),data_list[2],color_train_data_scatter]],name="TEST_AND_TRAIN_DATA_SCATTER"))
    summary_all_scatter = session.run(load_plot_to_tensorboard_x_plots_y_scatter([],[[range(len(data_list[1])),data_list[1],color_original_data_scatter],[range(window_size,window_size+len(data_list[2])),data_list[2],color_train_data_scatter],[range(window_size+train_range ,window_size+ train_range+ len(data_list[3])),data_list[3],color_test_data_scatter]],name="SUMMARIZE_ALL_SCATTER"))
    
    summary_list = [summary_original_unscaled,summary_original,summary_train,summary_original_with_train,summary_test,summary_test_with_train,summary_all,summary_original_scatter,summary_train_scatter,summary_original_with_train_scatter,summary_test_scatter,summary_test_with_train_scatter,summary_all_scatter]
    for i in summary_list:
        writer.add_summary(i)
    

In [4]:
def read_dataframe_and_output_dataset(name):
    tesla_stocks = pd.read_csv('AdidasAG10Years.csv')
    tesla_stocks.head()
    data_to_use = tesla_stocks['close'].values
    data_to_use = np.flipud(data_to_use) # flips values to get the newest values to the end
    return data_to_use
    

In [5]:
def scale_dataset(min,max,dataset): #dataset shape: [x]
    scaler = MinMaxScaler(feature_range=(-2,2))
    scaler = scaler.fit(dataset)
    scaled_dataset = scaler.transform(dataset)
    return scaler, scaled_dataset

In [6]:
def scale_dataset_back(scaler, dataset): #dataset shape: [x]
    return scaler.inverse_transform(dataset)

In [7]:
#if reshaped = False ->(None) to (None,1)
#if reshaped = True -> (None,1) to (None)
def reshape_dataset(dataset, reshaped):
    if reshaped is False:
        dataset = np.reshape(dataset, (-1,1))
    else:
        dataset = np.reshape(dataset, (-1))
    return dataset
    

In [8]:
# is needed because output from Training is [batch_size,1] and all appended : [number_of_batches,batch_size,1]
def unpack_predictions(prediction):
    sup =[]
    for i in range(len(prediction)):
        for j in range(len(prediction[i])):
            sup.append(prediction[i][j])
    return sup

In [9]:
def unpack_predictions_test(tests):
    test_results = []
    for i in range(len(tests)):
        for j in range(len(tests[i][0])):
            test_results.append(tests[i][0][j])
    return test_results

In [10]:
#requires dataset of shape (-1,1)
def window_data(reshaped_data, window_size):
    x = []
    y = []    
    i = 0
    while (i + window_size) <= len(reshaped_data) - 1: #heißt eigentlich nach windowsSize Tagen = eine Prediction
        x.append(reshaped_data[i:i+window_size])
        y.append(reshaped_data[i+window_size])        
        i += 1
    assert len(x) ==  len(y)
    return x, y

In [11]:
#requires windowed_dataset
def create_trains(windowed_data_x,windowed_data_y):
    x_train  = np.array(windowed_data_x[:train_range])
    y_train = np.array(windowed_data_y[:train_range])

    x_test = np.array(windowed_data_x[train_range:])
    y_test = np.array(windowed_data_y[train_range:])
    return x_train, y_train, x_test, y_test

In [12]:
def create_lstm_cell_with_MultiRNN_Dropout(neurons_at_gate, batch_size,number_of_lstm_layers,dropout_rate):
    layer_list = []
    for counter in range(number_of_lstm_layers):
        layer = tf.nn.rnn_cell.BasicLSTMCell(neurons_at_gate)
        if dropout_rate != 0:
            layer = tf.contrib.rnn.DropoutWrapper(layer, output_keep_prob=dropout_rate)
        layer_list.append(layer)     
    cell = tf.contrib.rnn.MultiRNNCell(layer_list) 
    init_state = cell.zero_state(batch_size, tf.float32) #zero_state -> alle values to 0
    return cell, init_state
    

In [13]:
# use for given accuracy function
#requires x shape of: 
def my_tf_round(x, decimal_place = 0):
    multiplier = tf.constant(10**decimal_place, dtype=x.dtype)
    return tf.round(x * multiplier) / multiplier

In [14]:
# requires logits and labels shape of: (batch_size,feature_size), normal_case: (8,1)
def accuracy(logits,labels,decimal_place):    
    targets_rounded = my_tf_round(labels,decimal_place)
    logits_rounded = my_tf_round(logits,decimal_place)
    correct_prediction=tf.equal(logits_rounded,targets_rounded)
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    return accuracy

In [15]:
def input_layer():
    x_batch = tf.placeholder(tf.float32, [batch_size, window_size, 1], name='input_data')
    y_batch = tf.placeholder(tf.float32, [batch_size, 1], name='targets')
    return x_batch,y_batch

In [16]:
def compute_rnn(x_batch,cell1,initial_state):
    output_dynamic_cell,states = tf.nn.dynamic_rnn(cell1, x_batch, initial_state=initial_state)
    return output_dynamic_cell, states

In [17]:
#lstm_output = (batch_size,window_size,neurons_at_gate)
#number of classes = often 1
#predictions_for_batch = (batch_size,number_of_features)
def output_layer(lstm_output, neurons_at_gate, number_of_features):  
    x = lstm_output[:, -1, :] 
    weights = tf.Variable(tf.truncated_normal([neurons_at_gate, number_of_features], stddev=0.05), name='output_layer_weights')
    bias = tf.Variable(tf.zeros([number_of_features]), name='output_layer_bias')
    predictions_for_batch = tf.matmul(x, weights) + bias 
    return predictions_for_batch

In [18]:
def loss(logits, targets): 
    losses = []
    for i in range(targets.get_shape()[0]):
        losses.append([(tf.pow(logits[i] - targets[i], 2))]) #mse      
    loss = tf.reduce_sum(losses)/(2*batch_size)
    return loss   

In [19]:
def optimize(loss,learning_rate,clip_on):
    gradients = tf.gradients(loss, tf.trainable_variables(),name="gradientsCalculated")
    clipper_, _ = tf.clip_by_global_norm(gradients, clip_on,name="ClipperIsSetTo4")
    optimizer = tf.train.AdamOptimizer(learning_rate,name="AdamOptimizer")
    train_optimizer = optimizer.apply_gradients(zip(gradients, tf.trainable_variables()),name="applyGradientsForOptimisation")
    return train_optimizer

In [20]:
def run_batch(session,x_batch,y_batch):
        logits_result,loss_result,accuracy_result,optimizer_result = session.run([logits,loss,accuracy,optimizer],feed_dict={x_batch_placeholder:x_batch,y_batch_placeholder:y_batch})
        return logits_result,loss_result,accuracy_result,optimizer_result
                                                                                                 

In [21]:
def run_batch_test(session,x_batch): #x_batch sollte eigentlich nur 1 window, mit dem man dann nicht nur 1 sondern x batches predicten kann
    logits_test_result = session.run([logits],feed_dict={x_batch_placeholder:x_batch})
    return logits_test_result

In [22]:
def run_all_epochs(session,x_train,y_train,dataset,scaled_dataset,summary_writer):
    
    
    epoch_loss = []
    accuracy = 0.0
    total_batch=0
    traind_scores = []
    
    for i in range(epochs):
        epoch_loss = []
        accuracy = 0.0
        total_batch=0
        traind_scores = [] 
        ii = 0
        while(ii + batch_size) <= len(x_train): #ein Durchlauf = 1 batch = 1 BPTT = 1 Anpassung der Gewichte = 1x "Lernen"
            x_batch = x_train[ii:ii+batch_size]
            y_batch = y_train[ii:ii+batch_size]
            logits,loss,accuracy,optimizer = run_batch(session,x_batch,y_batch)
            total_batch+=1
            ii += batch_size
            
            epoch_loss.append(loss)
            traind_scores.append(logits)
        if (i % 1) == 0:
            print('Epoch {}/{}'.format((i+1), epochs), ' Current loss: {}'.format(np.mean(epoch_loss)))
    
    prediction_list = unpack_predictions(traind_scores) #reason: from [number_of_batches,batch_size,1] to [-1]
    return prediction_list
    #dataset = scale_dataset_back(scaler, dataset)

In [23]:
def test_network(session,x_test,summary_writer):  
    tests = []
    i = 0
    while i+batch_size <= len(x_test): 
        logits_output = run_batch_test(session,x_test[i:i+batch_size])        
        i += batch_size
        tests.append(logits_output)
        
    result = unpack_predictions_test(tests)
    return result

In [24]:
def run_all(session):
    summary_writer = tf.summary.FileWriter(logs_path_test,session.graph)
    dataset=read_dataframe_and_output_dataset("AdidasAG10Years.csv")
    reshaped_data = reshape_dataset(dataset, False)
    scaler, scaled_dataset = scale_dataset(-1,1,reshaped_data)
    windowed_data_x, windowed_data_y = window_data(scaled_dataset, window_size)
    x_train, y_train, x_test, y_test = create_trains(windowed_data_x,windowed_data_y)
    
    prediction_list = run_all_epochs(session,x_train,y_train,dataset,scaled_dataset,summary_writer)
    result = test_network(session,x_test,summary_writer)
    
    create_and_add_summaries([dataset,scaled_dataset,prediction_list,result],writer=summary_writer,session=session) # if dataset changes, method have to be changed (updated)

    summary_writer.close()

In [25]:
window_size = 7 #arbitrarily selectable value
batch_size = 8
epochs = 1 #eig 3
train_range = 1600 #has to be dividable by batch_size
dropout_rate=0.00
number_of_lstm_layers= 1
learning_rate=0.001
neurons_at_gate=512
logs_path_test= 'logs/'
number_of_classes = 1 #ist glaube gleich mit number of features
decimal_place = 1
clip_on = 4
#number_of_features





x_batch_placeholder,y_batch_placeholder = input_layer()
cell, initial_state = create_lstm_cell_with_MultiRNN_Dropout(neurons_at_gate, batch_size, number_of_lstm_layers,dropout_rate)    
outputs, states =  compute_rnn(x_batch_placeholder,cell,initial_state=initial_state)  
logits = output_layer(outputs, neurons_at_gate, number_of_classes) 
loss = loss(logits,y_batch_placeholder)
accuracy = accuracy(logits,y_batch_placeholder,decimal_place)
optimizer= optimize(loss,learning_rate,clip_on)

In [26]:
###################################################
print("start")
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
run_all(sess)
sess.close()
print("finished")
###################################################

start
Epoch 1/1  Current loss: 0.013013721443712711
(912, 1)
finished
